In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import dask.dataframe as dd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score





/home/pimpas/Code/ml_transito/venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
data_folder = 'archive/'

csv_files = glob.glob(data_folder + '*.csv')


csv_files

['archive/Dados_PRF_2012.csv',
 'archive/Dados_PRF_2010.csv',
 'archive/Dados_PRF_2020.csv',
 'archive/Dados_PRF_2023.csv',
 'archive/Dados_PRF_2015.csv',
 'archive/Dados_PRF_2008.csv',
 'archive/Dados_PRF_2014.csv',
 'archive/Dados_Radares.csv',
 'archive/Dados_PRF_2009.csv',
 'archive/Dados_PRF_2021.csv',
 'archive/Dados_PRF_2019.csv',
 'archive/Dados_PRF_2007.csv',
 'archive/Dados_PRF_2013.csv',
 'archive/Dados_PRF_2016.csv',
 'archive/Dados_PRF_2017.csv',
 'archive/Dados_PRF_2018.csv',
 'archive/Dados_PRF_2011.csv',
 'archive/Dados_PRF_2022.csv']

In [3]:
csv_files.remove('archive/Dados_Radares.csv') 


In [4]:
csv_files

['archive/Dados_PRF_2012.csv',
 'archive/Dados_PRF_2010.csv',
 'archive/Dados_PRF_2020.csv',
 'archive/Dados_PRF_2023.csv',
 'archive/Dados_PRF_2015.csv',
 'archive/Dados_PRF_2008.csv',
 'archive/Dados_PRF_2014.csv',
 'archive/Dados_PRF_2009.csv',
 'archive/Dados_PRF_2021.csv',
 'archive/Dados_PRF_2019.csv',
 'archive/Dados_PRF_2007.csv',
 'archive/Dados_PRF_2013.csv',
 'archive/Dados_PRF_2016.csv',
 'archive/Dados_PRF_2017.csv',
 'archive/Dados_PRF_2018.csv',
 'archive/Dados_PRF_2011.csv',
 'archive/Dados_PRF_2022.csv']

In [5]:
# Dictionary to store DataFrames by year
df_years = {}

# Iterate over the files, read each CSV, and store it in the dictionary
for file in csv_files:
    # Extract the year from the filename using split
    year = file.split('_')[-1].split('.')[0]
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file, encoding='latin-1', delimiter=';')
    
    # Store the DataFrame in the dictionary with the year as the key
    df_years[year] = df

/tmp/ipykernel_38822/544647103.py:10: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='latin-1', delimiter=';')
/tmp/ipykernel_38822/544647103.py:10: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='latin-1', delimiter=';')
/tmp/ipykernel_38822/544647103.py:10: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='latin-1', delimiter=';')


In [6]:
df_years

{'2012':               id data_inversa dia_semana   horario  uf   br     km  \
 0        1035523   2012-01-01    Domingo  15:45:00  SC  282  513.7   
 1        1035527   2012-01-01    Domingo  03:00:00  SC  280    188   
 2        1035528   2012-01-01    Domingo  05:40:00  SC  116    1.5   
 3        1035530   2012-01-01    Domingo  06:00:00  RO  364  616.5   
 4        1035533   2012-01-01    Domingo  16:30:00  RJ  101     47   
 ...          ...          ...        ...       ...  ..  ...    ...   
 184563  83441270   2012-01-09    Segunda  11:30:00  CE  116   14.5   
 184564  83441274   2012-01-04     Quarta  08:00:00  CE  116   30.0   
 184565  83441277   2012-05-10     Quinta  18:00:00  CE  116    9.0   
 184566  83441278   2012-09-29     Sábado  06:10:00  CE  222  273.3   
 184567  83444176   2012-04-05     Quinta  20:00:00  CE  116  180.0   
 
                     municipio           causa_acidente  \
 0                       XAXIM  Velocidade incompatível   
 1                  

In [7]:
# Get the columns of the first dataframe
common_columns = set(df_years[list(df_years.keys())[0]].columns)

# Iterate over the remaining dataframes and find the common columns
for df_name in list(df_years.keys())[1:]:
    common_columns = common_columns.intersection(set(df_years[df_name].columns))

# Convert the set of common columns to a list
common_columns = list(common_columns)

In [8]:
df_years_common = {year: df[common_columns] for year, df in df_years.items()}

In [9]:
df_years_common

{'2012':         feridos_graves uso_solo     fase_dia condicao_metereologica   br  \
 0                    0    Rural    Pleno dia                    Sol  282   
 1                    0    Rural  Plena noite                  Chuva  280   
 2                    0   Urbano  Plena noite                  Chuva  116   
 3                    0    Rural    Pleno dia               Ignorada  364   
 4                    0    Rural  Plena noite                  Chuva  101   
 ...                ...      ...          ...                    ...  ...   
 184563               0   Urbano    Pleno dia                    Sol  116   
 184564               0    Rural    Pleno dia              Ceu Claro  116   
 184565               1   Urbano  Plena noite               Ignorada  116   
 184566               0    Rural    Pleno dia                    Sol  222   
 184567               0    Rural  Plena noite              Ceu Claro  116   
 
         ilesos   horario tracado_via  feridos_leves tipo_pista  .

In [10]:
common_columns

['feridos_graves',
 'uso_solo',
 'fase_dia',
 'condicao_metereologica',
 'br',
 'ilesos',
 'horario',
 'tracado_via',
 'feridos_leves',
 'tipo_pista',
 'uf',
 'classificacao_acidente',
 'tipo_acidente',
 'ignorados',
 'sentido_via',
 'data_inversa',
 'causa_acidente',
 'municipio',
 'dia_semana',
 'feridos',
 'veiculos',
 'id',
 'km',
 'pessoas',
 'mortos']

In [11]:
for df_name, df in df_years_common.items():
    print(df.isnull().sum())
    df.isnull().sum()


feridos_graves            0
uso_solo                  0
fase_dia                  0
condicao_metereologica    0
br                        0
ilesos                    0
horario                   0
tracado_via               0
feridos_leves             0
tipo_pista                0
uf                        0
classificacao_acidente    0
tipo_acidente             0
ignorados                 0
sentido_via               0
data_inversa              0
causa_acidente            0
municipio                 0
dia_semana                0
feridos                   0
veiculos                  0
id                        0
km                        0
pessoas                   0
mortos                    0
dtype: int64
feridos_graves            0
uso_solo                  0
fase_dia                  0
condicao_metereologica    0
br                        0
ilesos                    0
horario                   0
tracado_via               0
feridos_leves             0
tipo_pista                0
uf     

In [12]:
for df_name, df in df_years_common.items():
    df.dropna(inplace=True)
    print(df.isnull().sum())

/tmp/ipykernel_38822/3535963675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


feridos_graves            0
uso_solo                  0
fase_dia                  0
condicao_metereologica    0
br                        0
ilesos                    0
horario                   0
tracado_via               0
feridos_leves             0
tipo_pista                0
uf                        0
classificacao_acidente    0
tipo_acidente             0
ignorados                 0
sentido_via               0
data_inversa              0
causa_acidente            0
municipio                 0
dia_semana                0
feridos                   0
veiculos                  0
id                        0
km                        0
pessoas                   0
mortos                    0
dtype: int64
feridos_graves            0
uso_solo                  0
fase_dia                  0
condicao_metereologica    0
br                        0
ilesos                    0
horario                   0
tracado_via               0
feridos_leves             0
tipo_pista                0
uf     

In [13]:
for df_name, df in df_years_common.items():
    print(df.duplicated().sum())
    df.duplicated().sum()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [14]:
for year, df in df_years_common.items():
    df['year'] = year

combined_df = pd.concat(df_years_common.values(), ignore_index=True)

/tmp/ipykernel_38822/2962472929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = year
/tmp/ipykernel_38822/2962472929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = year
/tmp/ipykernel_38822/2962472929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [15]:
combined_df

,feridos_graves,uso_solo,fase_dia,condicao_metereologica,br,ilesos,horario,tracado_via,feridos_leves,tipo_pista,...,causa_acidente,municipio,dia_semana,feridos,veiculos,id,km,pessoas,mortos,year
0,0,Rural,Pleno dia,Sol,282,1,15:45:00,Curva,1,Simples,...,Velocidade incompatível,XAXIM,Domingo,1,1,1035523.0,513.7,2,0,2012
1,0,Rural,Plena noite,Chuva,280,2,03:00:00,Curva,1,Simples,...,Ingestão de álcool,MAFRA,Domingo,1,2,1035527.0,188,3,0,2012
2,0,Urbano,Plena noite,Chuva,116,1,05:40:00,Cruzamento,0,Simples,...,Velocidade incompatível,MAFRA,Domingo,0,1,1035528.0,1.5,1,0,2012
3,0,Rural,Pleno dia,Ignorada,364,0,06:00:00,Reta,1,Simples,...,Outras,ITAPUÃ DO OESTE,Domingo,1,1,1035530.0,616.5,1,0,2012
4,0,Rural,Plena noite,Chuva,101,0,16:30:00,Reta,1,Simples,...,Ingestão de álcool,CAMPOS DOS GOYTACAZES,Domingo,1,1,1035533.0,47,1,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012747,1,Sim,Plena Noite,Nublado,316.0,1,20:00:00,Reta,0,Dupla,...,Ausência de reação do condutor,PICOS,domingo,1,2,516543.0,313,4,0,2022
2012748,0,Sim,Plena Noite,Céu Claro,262.0,0,02:30:00,Curva,0,Simples,...,Velocidade Incompatível,VIANA,terça-feira,0,1,516703.0,17,1,0,2022
2012749,1,Não,Plena Noite,Chuva,146.0,0,19:00:00,Reta,1,Simples,...,Transitar na contramão,BOTELHOS,terça-feira,2,2,516707.0,459,2,0,2022
2012750,0,Sim,Pleno dia,Céu Claro,116.0,0,09:54:00,Reta,1,Múltipla,...,Manobra de mudança de faixa,FORTALEZA,domingo,1,2,516846.0,5,2,0,2022


In [16]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2012752 entries, 0 to 2012751
Data columns (total 26 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   feridos_graves          int64  
 1   uso_solo                object 
 2   fase_dia                object 
 3   condicao_metereologica  object 
 4   br                      object 
 5   ilesos                  int64  
 6   horario                 object 
 7   tracado_via             object 
 8   feridos_leves           int64  
 9   tipo_pista              object 
 10  uf                      object 
 11  classificacao_acidente  object 
 12  tipo_acidente           object 
 13  ignorados               int64  
 14  sentido_via             object 
 15  data_inversa            object 
 16  causa_acidente          object 
 17  municipio               object 
 18  dia_semana              object 
 19  feridos                 int64  
 20  veiculos                int64  
 21  id                      float64

In [17]:
# Set the target column
target = 'mortos'

# Split the dataframe into features and target
X = combined_df.drop([target], axis=1)
y = combined_df[target]

X = X.dropna()
y = y[X.index]

In [18]:
# Combine date and time into one datetime column
X['datetime'] = pd.to_datetime(X['data_inversa'] + ' ' + X['horario'], errors='coerce')

# Extract features from datetime
X['year'] = X['datetime'].dt.year
X['month'] = X['datetime'].dt.month
X['day'] = X['datetime'].dt.day
X['hour'] = X['datetime'].dt.hour
X['day_of_week'] = X['datetime'].dt.dayofweek

# Drop original date and time columns
X = X.drop(columns=['data_inversa', 'horario', 'datetime'])

X['br'] = pd.to_numeric(X['br'], errors='coerce')
X['km'] = pd.to_numeric(X['km'], errors='coerce')

X = X.dropna()
y = y[X.index]

In [19]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [20]:
# Initialize label encoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])


In [21]:
combined_df

,feridos_graves,uso_solo,fase_dia,condicao_metereologica,br,ilesos,horario,tracado_via,feridos_leves,tipo_pista,...,causa_acidente,municipio,dia_semana,feridos,veiculos,id,km,pessoas,mortos,year
0,0,Rural,Pleno dia,Sol,282,1,15:45:00,Curva,1,Simples,...,Velocidade incompatível,XAXIM,Domingo,1,1,1035523.0,513.7,2,0,2012
1,0,Rural,Plena noite,Chuva,280,2,03:00:00,Curva,1,Simples,...,Ingestão de álcool,MAFRA,Domingo,1,2,1035527.0,188,3,0,2012
2,0,Urbano,Plena noite,Chuva,116,1,05:40:00,Cruzamento,0,Simples,...,Velocidade incompatível,MAFRA,Domingo,0,1,1035528.0,1.5,1,0,2012
3,0,Rural,Pleno dia,Ignorada,364,0,06:00:00,Reta,1,Simples,...,Outras,ITAPUÃ DO OESTE,Domingo,1,1,1035530.0,616.5,1,0,2012
4,0,Rural,Plena noite,Chuva,101,0,16:30:00,Reta,1,Simples,...,Ingestão de álcool,CAMPOS DOS GOYTACAZES,Domingo,1,1,1035533.0,47,1,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012747,1,Sim,Plena Noite,Nublado,316.0,1,20:00:00,Reta,0,Dupla,...,Ausência de reação do condutor,PICOS,domingo,1,2,516543.0,313,4,0,2022
2012748,0,Sim,Plena Noite,Céu Claro,262.0,0,02:30:00,Curva,0,Simples,...,Velocidade Incompatível,VIANA,terça-feira,0,1,516703.0,17,1,0,2022
2012749,1,Não,Plena Noite,Chuva,146.0,0,19:00:00,Reta,1,Simples,...,Transitar na contramão,BOTELHOS,terça-feira,2,2,516707.0,459,2,0,2022
2012750,0,Sim,Pleno dia,Céu Claro,116.0,0,09:54:00,Reta,1,Múltipla,...,Manobra de mudança de faixa,FORTALEZA,domingo,1,2,516846.0,5,2,0,2022


In [22]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply standard scaling to the numerical columns
X = scaler.fit_transform(X)

In [23]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models in a dictionary
models = {
    'LinearRegression': LinearRegression(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    #'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'XGBRegressor': XGBRegressor()
}

In [24]:
# Initialize a dictionary to store results
results = {}

# Iterate over the models
for name, model in models.items():
    print(f"Training {name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store the results
    results[name] = {'MSE': mse, 'R2 Score': r2}
    
    # Print the evaluation metrics for each model
    print(f"{name} - MSE: {mse:.4f}, R2 Score: {r2:.4f}")

# Convert results into a DataFrame for easier visualization
results_df = pd.DataFrame(results).T
print("\nModel Performance Comparison:")
print(results_df)

Training LinearRegression...


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets